In [1]:
import numpy as np
from itertools import groupby
from functools import partial

## Radix-n data streamlining

For a radix-2 FFT, it is possible to divide input data in such a way that every butterfly operation reads and writes to two different arrays in memory. This could be advantageous for an implementation on the FPGA. The trick is to separate data locations based on the parity of their index.

### Parity

The index of each element in the array can be written in binary. In the case of a radix-2 FFT, we can see the array being reshaped in a $[2, 2, 2, \dots]$ shape. Every stage of the $n=2^k$ sized radix-2 FFT is being performed in a different dimension of the $k$-dimensional reshaped array. The indexing in this multi-dimensional array is the same as the binary notation for the linear index. That means that each time the 2-FFT is performed, the multi-index of the elements involved will differ by one bit in the linear index. Separating the array based on the parity of the index will guarantee that each 2-FFT operation reads and writes one (complex) number from each set. The parity is the sum of all the individual bits, modulo 2.

$$P_2(i) = \left(\sum_k b_k\right) \mod 2,\quad {\rm where}\, i := \sum_k b_k 2^k$$

We can extend this concept to the radix-4 FFT, on 4 data channels.

### 4-parity

In the radix-4 FFT we can view the array as being reshaped to a $[4, 4, 4, \dots]$ shape. The multi-index into this array is equivalent to the quarternary number notation of the linear index. Similar to the radix-2 parity, we can define the radix-4 parity as the sum of each quarternary digit in the index, modulo 4.

$$P_4(i) = \left(\sum_k q_k\right) \mod 4,\quad {\rm where}\, i := \sum_k q_k 4^k$$

This would ensure that each 4-FFT reads its data from the 4 different input channels. We define the `parity` function to work with any radix:

In [2]:
def digits(n, i):
    """Generates the n-numbered digits of i, in reverse order."""
    while True:
        if i == 0:
            return
        else:
            i, q = divmod(i, n)
            yield q
            
def parity(n, i):
    return sum(digits(n, i)) % n

To see which index belongs in which channel, we use a `groupby` on keyed the `parity` function

In [3]:
def channels(N, radix):
    parity_r = partial(parity, radix)
    return groupby(sorted(range(N), key=parity_r), parity_r)

for instance, for radix-2, size 16, this creates the following channels

In [4]:
for (g, i) in channels(16, 2):
    print(g, list(i))

0 [0, 3, 5, 6, 9, 10, 12, 15]
1 [1, 2, 4, 7, 8, 11, 13, 14]


### Example: radix-2 16-FFT

We can implement the 16-FFT as a radix-2 operation on an array of shape $[2,2,2,2]$.

In [5]:
def fft2(x0, x1):
    y = np.zeros(shape=(2,), dtype='complex64')
    y[0] = x0 + x1
    y[1] = x0 - x1
    return y

To check whether our grouping works, we create a dummy array, and a dummy `fft2_check` that just checks if all arguments are in different channels.

In [6]:
def fft2_check(x0, x1):
    assert np.all(x0 != x1)

In [7]:
x = np.zeros(shape=(16,), dtype=int)
for (g, i) in channels(16, 2):
    x[list(i)] = g

In [8]:
factors = [2, 2, 2, 2]
s = x.reshape(factors)

try:
    for k in range(len(factors)):
        fft2_check(np.take(s, 0, axis=k), np.take(s, 1, axis=k))
except AssertionError:
    print("failed")
else:
    print("succeeded")

succeeded


### Example: radix-4 64-FFT

In [9]:
for g, i in channels(64, 4):
    print(g, list(i))

0 [0, 7, 10, 13, 19, 22, 25, 28, 34, 37, 40, 47, 49, 52, 59, 62]
1 [1, 4, 11, 14, 16, 23, 26, 29, 35, 38, 41, 44, 50, 53, 56, 63]
2 [2, 5, 8, 15, 17, 20, 27, 30, 32, 39, 42, 45, 51, 54, 57, 60]
3 [3, 6, 9, 12, 18, 21, 24, 31, 33, 36, 43, 46, 48, 55, 58, 61]


In [10]:
from itertools import combinations
import math

def fft4_check(x0, x1, x2, x3):
    for (a, b) in combinations([x0, x1, x2, x3], 2):
        assert np.all(a != b)

In [11]:
N = 64
radix = 4

x = np.zeros(shape=(N,), dtype=int)
for g, i in channels(N, radix):
    x[list(i)] = g

In [12]:
factors = [radix] * int(math.log(N, radix))
s = x.reshape(factors)

try:
    for k in range(len(factors)):
        fft4_check(*(np.take(s, i, axis=k) for i in range(radix)))
except AssertionError:
    print("failed")
else:
    print("succeeded")

succeeded
